In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv('/home/wendyunji/MLAC/data/Optc Data/bro_0923_labeled_long_fin.csv')

In [18]:
#
# data.drop_duplicates(inplace=True, keep=False, ignore_index=True)
#
data.dropna(axis=0, inplace=True, how="any")
#
data.replace([-np.inf, np.inf], np.nan, inplace=True)
data.dropna(axis=0, how='any', inplace=True)
#
# data_std =data.std(numeric_only=True)
# threshold=0.01
# constant_features = [column for column, std in data_std.iteritems() if std < threshold]
# data.drop(labels=constant_features, axis=1, inplace=True)
#
# threshold=0.98
# data_corr = data.corr()
# mask = np.triu(np.ones_like(data_corr, dtype=bool))
# tri_df = data_corr.mask(mask)
# correlated_features = [c for c in tri_df.columns if any(tri_df[c] > threshold)]
#data.drop(labels=correlated_features, axis=1, inplace=True)

In [19]:
data.head()

,Unnamed: 0,duration,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,conn_state_REJ,conn_state_RSTO,conn_state_RSTR,...,history_ShAdDafFR,history_ShAdDatFf,history_ShAdDatfF,history_ShAdDtaFf,history_ShAdDtafF,history_ShDAadfF,history_ShDadAfF,history_ShR,history_Sr,history_^fA
0,0,0.011354,10,1382,8,1084,0.0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.011485,10,1382,8,1084,0.0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.009323,11,1614,8,1196,0.0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.017738,10,1398,8,1132,0.0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.017420,11,1470,8,1116,0.0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
y = data['label']
y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: label, dtype: float64

In [21]:
X = data.drop('Unnamed: 0', axis=1)
X = X.drop('label', axis=1)
X.columns

Index(['duration', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes',
       'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTR', 'conn_state_S0',
       'conn_state_SF',
       ...
       'history_ShAdDafFR', 'history_ShAdDatFf', 'history_ShAdDatfF',
       'history_ShAdDtaFf', 'history_ShAdDtafF', 'history_ShDAadfF',
       'history_ShDadAfF', 'history_ShR', 'history_Sr', 'history_^fA'],
      dtype='object', length=144)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(len(X_test))
print(len(X_train))

1269808
2962883


In [23]:
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

In [24]:
models = []
#models.append(('SVM', SVC(gamma='auto')))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier(max_depth=5)))
models.append(('RF', RandomForestClassifier(max_depth=5, n_estimators=5, max_features=3)))    
models.append(('ABoost', AdaBoostClassifier()))
models.append(('LR', LogisticRegression(solver='lbfgs', max_iter=200)))
models.append(('NB', GaussianNB()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('MLP', MLPClassifier()))

In [25]:
print('Model\tAcc\tF1_mi\tRecall_mi\tPrecision_mi\tF1_ma\tRecall_ma\tPrecision_ma\tF1_we\tRecall_we\tPrecision_we\tExecution')

for name, model in models:
    start_time = time.time()

    model.fit(X_train, y_train)
    prediction = model.predict(X_test)

    delta = time.time() - start_time
    acc = accuracy_score(y_test, prediction)
    f1_mi = f1_score(y_test, prediction,average='micro')
    f1_ma = f1_score(y_test, prediction,average='macro')
    f1_we = f1_score(y_test, prediction,average='weighted')
    recall_mi = recall_score(y_test, prediction, average='micro')
    recall_ma = recall_score(y_test, prediction, average='macro')
    recall_we = recall_score(y_test, prediction, average='weighted')
    precision_mi = recall_score(y_test, prediction, average='micro')
    precision_ma = recall_score(y_test, prediction, average='macro')
    precision_we = recall_score(y_test, prediction, average='weighted')
    print('{}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.2f} secs'.format(name, acc, f1_mi, recall_mi, precision_mi, f1_ma, recall_ma, precision_ma, f1_we, recall_we, precision_we, delta))

Model	Acc	F1_mi	Recall_mi	Precision_mi	F1_ma	Recall_ma	Precision_ma	F1_we	Recall_we	Precision_we	Execution
KNN	1.000	1.000	1.000	1.000	1.000	1.000	1.000	1.000	1.000	1.000	1435.29 secs
CART	1.000	1.000	1.000	1.000	1.000	1.000	1.000	1.000	1.000	1.000	4.03 secs
RF	1.000	1.000	1.000	1.000	1.000	1.000	1.000	1.000	1.000	1.000	6.01 secs
ABoost	1.000	1.000	1.000	1.000	1.000	1.000	1.000	1.000	1.000	1.000	5.44 secs


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0.0